# Demo - Print Evaluation Steps

The macro `@print_eval_steps` can evaluate an expression step-by-step and print out all the intermediate steps.

## Definition

In [1]:
macro print_eval_steps(expr)
    initial_expr = expr
    counter = 1
    
    println("Initial: $expr")
    
    while typeof(expr) == Expr
        expr = eval_next_step(expr)
        println("Step $counter: $expr")
        counter += 1
    end
    
    println("Result: $initial_expr evaluates to $expr")
end

@print_eval_steps (macro with 1 method)

In [2]:
function eval_next_step(expr::Expr)
    if expr.head ∈ (:call, :.)
        for (i, arg) in enumerate(expr.args)
            if typeof(arg) == Expr
                return Expr(expr.head, replace_value(expr.args, eval_next_step(arg), i)...)
            end
        end
    end

    return eval(expr)
end

eval_next_step(others) = others

eval_next_step (generic function with 2 methods)

In [3]:
replace_value(v::Vector, value, index::Int) = [(i == index ? value : v[i]) for i in 1:length(v)]

replace_value (generic function with 1 method)

## Examples

Here are some example expression that the macro can work on.

In [4]:
@print_eval_steps 4 + 5 * 6 + 2 ÷ 2

Initial: 4 + 5 * 6 + 2 ÷ 2
Step 1: 4 + 30 + 2 ÷ 2
Step 2: 4 + 30 + 1
Step 3: 35
Result: 4 + 5 * 6 + 2 ÷ 2 evaluates to 35


In [5]:
@print_eval_steps sin((1 + sqrt(5)) / 2)

Initial: sin((1 + sqrt(5)) / 2)
Step 1: sin((1 + 2.23606797749979) / 2)
Step 2: sin(3.23606797749979 / 2)
Step 3: sin(1.618033988749895)
Step 4: 0.9988845090948848
Result: sin((1 + sqrt(5)) / 2) evaluates to 0.9988845090948848
